# Gibbs measure over a grid tutorial

This notebook illustrates the knockoff construction for a Gibbs measure across a grid with probability mass function
$$p(X) = \frac{1}{Z(\beta_0)} \exp\left(-\beta_0\sum_{\substack{s, t \in \mathcal I \\\|s-t\|_1=1}}(x_s-x_t)^2 \right),\quad\mathcal I = \{(i_1,i_2) : 1\le i_1,i_2 \le d\}.$$
Each variable has support $\{0,\dots,K\}$ where $K$ is a positive integer.

## Multiple-try Metropolis

For large $K$, Multiple-try Metropolis gives the most efficient sampler for this class of distributions. We demonstrate this technique below.

In [1]:
import math
import numpy as np

%run ../Gibbs-grid/Gibbs_MTM #load the Gibbs simulation functions

In [2]:
#simulation parameters
d = 5
p = d * d #total number of variables
K = 5 # support size
beta = 0.2 #interaction strength
numsamples = 100

#algorithm parameters
N_gibbs = 100 #number of iterations for the initial Gibbs sampler
gamma = .99 
half_num_try = 1 #half the number of proposals
step_size = 1 #propsal size
max_width = 2 #slicing width

We use a Gibbs sampler to generate samples from the model, and then generate a corresponding knockoff.

In [3]:
bigmatrix = np.zeros([numsamples,2*p]) #store simulation results
for i in range(numsamples):
    x_obs = Gibbs_sampler(d, d, K, beta, N = N_gibbs)
    xk = SCEP_MH_Gibbs_slice(k1 = d, k2 = d, K = K, 
        beta = beta, x_obs = x_obs,  gamma = gamma, 
        half_num_try = half_num_try, step_size = step_size, 
        max_width = max_width)
    bigmatrix[i,:] = np.append(np.reshape(x_obs, p), np.reshape(xk, p))
    
np.shape(bigmatrix)

(100, 50)

We can evaluate the quality of these knocokffs by computing the average correlation between $X_{i,j}$ and $\tilde{X}_{i,j}$.

In [4]:
cors = []
for j in range(p):
    cors += [np.corrcoef(bigmatrix[:, j].T, 
                         bigmatrix[:, j + p].T)[0,1]]
np.mean(cors)     

0.7109167017670421